In [1]:
import numpy as np
from pypianoroll import *
from matplotlib import pyplot as plt
import os
import sys
import time
import tensorflow as tf

In [2]:
# iteriere durch alle Ordner 
# Code basiert auf https://www.tutorialspoint.com/python/os_walk.htm 
# 21,425 Lieder; für training nur 500 bis 2000 nötig. sonst zu wenig wiederholungen beim training! 
def Crawler(path, Durchlauf):
    Song_Array = []
    skip = 0
    for root, dirs, files in os.walk(path):
        if len(Song_Array) >= 100:
            break
        for name in files:
            multitrack = Multitrack(os.path.join(root, name)) 
            multitrack.binarize()
            for i in range(17):
                if len(multitrack.tracks[i].pianoroll) > 2280:
                    skip += 1
                    if skip > (Durchlauf*100):
                        Song_Array.append(multitrack)
                    break

            # print(os.path.join(root, name))
        
    return Song_Array

In [3]:
def Training_Data_Adaption(SongArray, save):
    # TrainDataArray[len(SongArray)][6][4][96][84][17]
    TrainDataArray = []
    for s in range(len(SongArray)):                   # Liedazahl
        Schnitt = []
        for l in range(6):                     # Liedausschnitte
            Tackt = []
            for t in range(4):                   # Tackte
                Schritt = []
                for z in range(96):            # Zeitschritte
                    pitch = []
                    for p in range(84):          # pitch_nr
                        instrument = []
                        for i in range(17):          # Instrumente
                            if len(SongArray[s].tracks[i].pianoroll) != 0:
                                instrument.append(SongArray[s].tracks[i].pianoroll[(z + 96*t + 96*4*l)][(p+24)]) # append true/falsus
                            else :
                                instrument.append(False) # append true/false
                        pitch.append(instrument)
                    Schritt.append(pitch)
                Tackt.append(Schritt)
            Schnitt.append(Tackt)
        TrainDataArray.append(Schnitt)
        SongArray[s] = None
    # save TrainArray        
    np.save(save, TrainDataArray, allow_pickle=True)
    SongArray = []

In [4]:
#iteriere durch trainingsdaten und verbinde zu einem größeren Datensatz
def combine_Data(Durchlaeufe):
    Final = []
    tmp = []
    for Durchlauf in range(Durchlaeufe): 
        loadFile = os.path.join(".", "training_data", "lpd_17_{}_cleansed.npy".format(Durchlauf))
        tmp = np.load(loadFile, allow_pickle=True)
        Final.extend(tmp)
    save = os.path.join(".", "training_data", "lpd_17_Final_{}_cleansed.npy".format((Durchlaeufe*100)))
    np.save(save, Final, allow_pickle=True)
        

In [5]:
#starte alles
def main(Durchlaeufe):
    #start-pfad
    file = os.path.join(".", "training_data", "lpd_17_cleansed")
    for Durchlauf in range(Durchlaeufe):
        save = os.path.join(".", "training_data", "lpd_17_{}_cleansed.npy".format(Durchlauf))
        #start crawler
        Array = []
        Array = Crawler(file, Durchlauf)
        # start Train Data transformation and save
        Training_Data_Adaption(Array, save)
    combine_Data(Durchlaeufe)

In [6]:
# start to prepare training Data
Durchlaeufe = 5 #x100 lieder in den Trainingsdatensatz (datensatz besteht aus >20k Lieder, alle kürzeren werden übersprungen)
main(Durchlaeufe)